In [1]:
# Import libraries
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import dlib
from imutils import face_utils
from keras.models import load_model
from datetime import date
from datetime import datetime
from time import time, sleep
import joblib
# from firebase import firebase

In [3]:
#import pre-trained models to recognize face area and landmarks in the face 
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
landmark_detector=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [4]:
#Import trained models
emotion_model = load_model('models-44')
ear_model=load_model('EAR.model')
arousal_model = load_model('models_ar-296')
valence_model = load_model('models_va-229')

In [5]:
ar_scaler=joblib.load('arl-scaler.sav')
vl_scaler=joblib.load('val-scaler.sav')

In [6]:
emotion_labels = ['angry','happy', 'sad', 'surprise', 'neutral']
ear_dict={0:'alert',1: 'drowsy'}

face_detector=dlib.get_frontal_face_detector()

window_size=10

In [7]:
#Array initialization
timeList = []
dateList = []
predEmotions = []
predSleepiness = []
predVal = []
predAr = []

In [8]:
currentDate_1 = date.today()

In [9]:
#Create CSV file
def FerDataFrame():
    dictData = {'Date':dateList, 'Time': timeList, 'Emotion Category': predEmotions, 'Sleepiness Stage':predSleepiness, 'Valence':predVal,'Arousal':predAr }
    
    dfData = pd.DataFrame(dictData)
    print(dfData)

    dfData.to_csv(str(currentDate_1) + '_FERData.csv')

In [10]:
# def sleepinessTimeFrame():
#     dictsleepiness = {'Date':dateList,'Time':timeList, 'Sleepiness Stage': predSleepiness}
#     dfSleepiness = pd.DataFrame(dictsleepiness)
#     print(dfSleepiness)
# #     firebase1 = firebase.FirebaseApplication('https://cigapp-d83d9-default-rtdb.asia-southeast1.firebasedatabase.app/', None)
# #     result = firebase1.post('Parent/-Ml5UkMGSy1hCModh2Fs/Children/-Ml5V5GB9L1RvGvp0Bf8/sleepiness/', dictsleepiness)
#     dfSleepiness.to_csv(str(currentDate_1) + '_Sleepiness.csv')

In [11]:
#Realtime emotion detection code
def realtimeDetection():
    
    #Initialization
    test_data=[]
    ar_vals=[]
    vl_vals=[]
    count=0
    sleepLabel='None'
    bestEmotion='None'
    bestAr=0
    bestVl=0
    currentTime = 0
    emotionList = []

    start_now = datetime.now()
    start = start_now.strftime("%H:%M:%S")

    cap=cv2.VideoCapture(0)
    while(True):
        ret,img=cap.read()

        if(ret==False):
            break
        img=cv2.resize(img,(0,0),fx=0.5,fy=0.5)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces=face_detector(gray)

        for face in faces:

            x1=face.left()
            y1=face.top()
            x2=face.right()
            y2=face.bottom()

            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)

            roi_gray = gray[y1:y2,x1:x2]
            
            test_img=cv2.resize(roi_gray,(48,48))/255
            
            roi_gray = cv2.resize(roi_gray,(100,100),interpolation=cv2.INTER_AREA)
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            preds = emotion_model.predict(roi)[0]
            expLabel=emotion_labels[np.argmax(preds)] 
            emotionList.append(expLabel)   
            
            test_img=test_img.reshape(1,48,48,1)
            ar_result=arousal_model.predict(test_img)
            ar_result=ar_scaler.inverse_transform(ar_result)[0][0]
            ar_vals.append(ar_result)
            
            vl_result=valence_model.predict(test_img)
            vl_result=vl_scaler.inverse_transform(vl_result)[0][0]
            vl_vals.append(vl_result)
            
            points=landmark_detector(gray,face)

            points=face_utils.shape_to_np(points)

            for i,p in enumerate(points):

                cen=(p[0],p[1])
                cv2.circle(img,cen,2,(0,255,0),-1)

            cv2.line(img,tuple(points[37]),tuple(points[41]),(0,0,255),2)
            cv2.line(img,tuple(points[44]),tuple(points[46]),(0,0,255),2)

            p1=points[36][0]
            p2=points[37][1]
            p3=points[38][1]
            p4=points[39][0]
            p5=points[40][1]
            p6=points[41][1]

            eye_ratio=(abs(p2-p6)+abs(p3-p5))/abs(p1-p4)
            test_data.append(eye_ratio)
            count=count+1

            if(count==window_size):

                test_data=np.array(test_data)
                test_data=test_data.reshape(1,test_data.shape[0],1)
                result=ear_model.predict(test_data)
                label=np.argmax(result,axis=1)[0]
               
                sleepLabel=ear_dict[label]
                bestEmotion=max(emotionList,key=emotionList.count)
                bestAr=round(sum(ar_vals)/len(ar_vals),2)
                bestVl=round(sum(vl_vals)/len(vl_vals),2)

                now = datetime.now()

                currentTime = now.strftime("%H:%M:%S")
                currentDate = date.today()
    
                dateList.append(currentDate)
                timeList.append(currentTime)       
                predEmotions.append(bestEmotion)
                predSleepiness.append(sleepLabel)
                predVal.append(bestVl)
                predAr.append(bestAr)
         
                count=0
                test_data=[]
                emotionList=[] 
                ar_vals=[]
                vl_vals=[]
                
        FerDataFrame()
        cv2.putText(img,sleepLabel,(10,50),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2) 
        cv2.putText(img,bestEmotion,(10,100),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),3)
        cv2.putText(img,'AR:'+str(bestAr),(10,150),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),3) 
        cv2.putText(img,'VL:'+str(bestVl),(10,200),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),3) 
           
        cv2.imshow('LIVE',img)
        
        k=cv2.waitKey(1)
        if(k==27):
            break
    cv2.destroyAllWindows()


In [ ]:
#Schedule to run everyday at 12am
import schedule

schedule.every().day.at("00:00").do(realtimeDetection())

while 1:
    schedule.run_pending()
    time.sleep(1)